In [1]:
# Inception结构是一种和LeNet-5结构完全不同的卷积神经网络结构，在LeNet-5模型中，不同卷积层通过串联的方式连接在一起，
# 而Inception-v3模型中的Inception结构是将不同的卷积层通过并联的方式结合在一起
# Inception-v3模型总共有46层，由11个Inception模块组成

# 一、 tf-slim实现卷积层示例

In [ ]:
import tensorflow as tf

# 如果按照LeNet-5方式实现Inception-v3，那么代码量会非常大，可读性会非常差
# 所以需要引用TensorFlow-Slim工具类更加简洁的实现一个卷积层

# 直接使用TensorFlow原始API实现卷积层
with tf.variable_scope(scope_name):
    weights = tf.get_variable("weight", ...)
    biases = tf.get_variable("bias", ...)
    conv = tf.nn.conv2d(...)
    relu = tf.nn.relu(tf.nn.bias_add(conv, biases))
    
# 使用TensorFlow-Slim实现卷积层
net = slim.conv2d(input, 32, [3,3])

# 二、tf-slim实现Inception模块

In [3]:
import tensorflow as tf

In [ ]:
# 加载slim库
slim = tf.contrib.slim

# slim.arg_scope函数可以用于设置默认的参数取值
with slim.arg_scope([slim.conv2d, slim.max_pool2d, slim.avg_pool2d], stride = 1, padding = 'VALID'):
    # 假设输入图片经过之前的神经网络前向传播的结果保存在变量net中
    net = 上一层的输出节点矩阵
    
    # 为一个Inception模块声明一个统一的变量命名空间
    with th.variable_scope('Mixed_7c'):
        # 给Inception模块中每一条路径声明一个命名空间
        with tf.variable_scope('Branch_0'):
            branch_0 = slim.conv2d(net, 320, [1,1], scope = 'Conv2d_0a_1*1')
        
        # Inception模块中的第二条路径
        # 这条计算路径上的结构本身也是一个Inception结构
        with tf.variable_scope('Branch_1'):
            branch_1 = slim.conv2d(net, 384, [1,1], scope = 'Conv2d_0a_1*1')
            # tf.contact可以将多个矩阵拼接起来
            branch_1 = tf.concat(3, [
                # 此处二层卷积层的输入是branch_1,而不是net
                slim.conv2d(branch_1, 384, [1,3], scope = 'Conv2d_0b_1*3')
                slim.conv2d(branch_1, 384, [3,1], scope = 'Conv2d_0c_3*1')
            ])
            
        # Inception 模块中第三条路径，此计算路径也是一个Inception结构
        with tf.variable_scope('Branch_2'):
            branch_2 = slim.conv2d(
                net, 448, [1,1], scope = 'Conv2d_0a_1*1')
            branch_2 = slim.conv2d(
                branch_2, 384, [3,3], scope = 'Conv2d_0b_3*3')
            branch_2 = tf.concat(3, [
                # 此处二层卷积层的输入是branch_1,而不是net
                slim.conv2d(branch_2, 384, [1,3], scope = 'Conv2d_0c_1*3')
                slim.conv2d(branch_2, 384, [3,1], scope = 'Conv2d_0c_3*1')
            ])
            
        # Inception模块中第四条路径
        with tf.variable_scope('Branch_3'):
            branch_3 = slim.avg_pool2d(
                net, [3,3], scope = 'AvgPool_0a_3*3')
            branch_3 = slim.conv2d(
                branch_3, 192, [1,1], scope = 'Conv2d_0b_1*1')
            
        # 当前Inception模块的最后输出是由上面4个计算结果拼接得到的
        net = tf.concat(3, [branch_0, branch_1, branch_2, branch_3])